In [1]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd
%load_ext autoreload
%autoreload 2

## Load the training data into feature matrix, class labels, and event ids:

In [2]:
from proj1_helpers import *
DATA_TRAIN_PATH = os.path.join(os.getcwd(), '../data/train.csv')
y, tX, ids = load_csv_data(DATA_TRAIN_PATH)

## Our CRAZY machine learning thing :) ...

In [3]:
from nn_model import NNModel
from preprocessing import *
from nn_model import *
import pandas as pd

In [4]:
preprocessing = Preprocessing()
y[y==-1] = 0
y = y.astype(np.int)
preprocessing.replace_outliers_by_nan(tX)

#Delete rows from tX and y, which has more than 8 nan values in tX and 0 label in y
num_nan_in_row = np.isnan(tX).sum(axis=1)
mask = (y==0) & (num_nan_in_row>8)
y = y[~mask]
tX = tX[~mask]

In [12]:
def hyperparamters_tunning(x_train, y_train,
                           x_val, y_val,
                           use_transformations, handling_outliers, degrees,
                           lrs, lambdas, epochs, batch_size):
    best = {
        "models_feat": {},
        "model": None,
        "transformation": None,
        "handling_outlier": None,
        "degree": None,
        "accuracy": 0,
        "lr": None,
        "lambda": None,
    }
    models = {}
    model = None
    units = 1
    activation = 'sigmoid'
    for use_transformation in use_transformations:
        for handling_outlier in handling_outliers:
            for lr in lrs:
                for lambda_ in lambdas:
                    for degree in degrees:
                        preprocessing = Preprocessing(
                            use_transformations=use_transformation,
                            handling_outliers=handling_outlier,
                        )
                        train_data, models = preprocessing.preprocess(
                            x_train,
                            lr=lr,
                            lambda_=lambda_,
                            batch_size=batch_size,
                            epochs=epochs,
                            degrees=degree
                        )
                        
                        for lr in lrs:
                            for lamda_ in lambdas:
                                model = NNModel(train_data.shape[1])
                                model.add_layer(1, activation='sigmoid')
                                model.train(train_data, y_train, lr=lr,
                                            lambda_=lambda_,
                                            batch_size=batch_size, epochs=100,
                                            verbose=1)
                                preprocessing = Preprocessing(
                                    use_transformations=use_transformation,
                                    handling_outliers = handling_outlier,
                                )
                                val_data, _ = preprocessing.preprocess(
                                    x_val,
                                    train=False,
                                    models=models,
                                    degrees=degree
                                )
                                
                                y_pred = model.predict(val_data)
                                y_pred = y_pred > 0.5
                                y_pred = y_pred.squeeze()
                                accuracy = (y_pred==y_val).mean()
                                if accuracy > best["accuracy"]:
                                    print("Best is updated with accuracy:", accuracy)
                                    best["models_feat"] = models
                                    best["model"] = model
                                    best["transformation"] = use_transformation
                                    best["handling_outlier"] = handling_outlier
                                    best["degree"] = degree
                                    best["accuracy"] = accuracy
                                    best["lambda"] = lambda_
                                    best["lr"] = lr
    return best

In [13]:
num_examples = tX.shape[0]
val_size = 10000
train_samples = np.random.choice(num_examples, replace=False, size=val_size)
mask_val = np.isin(np.arange(tX.shape[0]), train_samples)
x_val, y_val = tX[mask_val], y[mask_val]
x_train, y_train = tX[~mask_val], y[~mask_val]

In [14]:
use_transformations = [False]
handling_outliers = ['predict', 'fill_mean', 'remove']
lrs = [0.01, 0.1]
lambdas = [0.3, 1, 10]
degrees = [np.arange(2, 5)]
epochs = 100
batch_size = 64

In [15]:
best = hyperparamters_tunning(x_train, y_train,
                              x_val, y_val,
                              use_transformations=use_transformations,
                              handling_outliers=handling_outliers,
                              degrees=degrees,
                              lrs=lrs,
                              lambdas=lambdas,
                              epochs=epochs,
                              batch_size=batch_size
                             )

Preprocesing started!

Training started
>Epoch #100:	[####################]; Loss: 7.846122817948441
Training ended

Training started
>Epoch #100:	[####################]; Loss: 10.83490184831185
Training ended

Training started
>Epoch #100:	[####################]; Loss: 9.113505384771914
Training ended

Training started
>Epoch #100:	[####################]; Loss: 11.522920802590024
Training ended

Training started
>Epoch #100:	[####################]; Loss: 13.116643339484433
Training ended

Training started
>Epoch #100:	[####################]; Loss: 7.6213049455049235
Training ended

Training started
>Epoch #100:	[####################]; Loss: 469.47625688557133
Training ended

Training started
>Epoch #100:	[####################]; Loss: 9.846582418795833
Training ended

Training started
>Epoch #100:	[####################]; Loss: 10.853308414534204
Training ended

Training started
>Epoch #100:	[####################]; Loss: 13.88827262796391
Training ended

Training started
>Epoch #100:	[#

KeyboardInterrupt: 

In [101]:
model = best["model"]
best["models_feat"] = models
use_transformation = best["transformation"]
handling_outlier = best["handling_outlier"]
degree = best["degree"]
accuracy = best["accuracy"]
lambda_ = best["lambda"]
lr = best["lr"]

# Looking in data

In [58]:
import pandas as pd
import os


data = pd.read_csv('../data/train.csv')
data = data.drop(columns=['Id'])
mask = (data.loc[:, 'Prediction'] == 's').values
data.loc[mask, ('Prediction')] = 1
data.loc[~mask, ('Prediction')] = 0
data = data.astype({'Prediction': np.int})

data[data==-999] = np.nan

In [59]:
%matplotlib notebook
data.iloc[:, :15].hist()
data.iloc[:, 15:].hist()

array([[<matplotlib.axes._subplots.AxesSubplot object at 0x7fd83b241250>,
      dtype=object)

In [56]:
%matplotlib notebook
mask = 
data.iloc[data., :15].hist()
data.iloc[:, 15:].hist()

,DER_mass_MMC,DER_mass_transverse_met_lep,DER_mass_vis,DER_pt_h,DER_deltaeta_jet_jet,DER_mass_jet_jet,DER_prodeta_jet_jet,DER_deltar_tau_lep,DER_pt_tot,DER_sum_pt,...,PRI_met_phi,PRI_met_sumet,PRI_jet_num,PRI_jet_leading_pt,PRI_jet_leading_eta,PRI_jet_leading_phi,PRI_jet_subleading_pt,PRI_jet_subleading_eta,PRI_jet_subleading_phi,PRI_jet_all_pt
count,211886.000000,250000.000000,250000.000000,250000.000000,72543.000000,72543.000000,72543.000000,250000.000000,250000.000000,250000.000000,...,250000.000000,250000.000000,250000.000000,150087.000000,150087.000000,150087.000000,72543.000000,72543.000000,72543.000000,250000.000000
mean,121.858528,49.239819,81.181982,57.895962,2.403735,371.783360,-0.821688,2.373100,18.917332,158.432217,...,-0.010119,209.797178,0.979176,84.822105,-0.003275,-0.012393,57.679474,-0.011845,-0.001582,73.064591
std,57.298157,35.344886,40.828691,63.655682,1.742226,397.699325,3.584362,0.782911,22.273494,115.706115,...,1.812223,126.499506,0.977426,60.662276,1.784546,1.813385,31.985782,2.031743,1.816950,98.015662
min,9.044000,0.000000,6.329000,0.000000,0.000000,13.602000,-18.066000,0.208000,0.000000,46.104000,...,-3.142000,13.678000,0.000000,30.000000,-4.499000,-3.142000,30.000000,-4.500000,-3.142000,0.000000
25%,91.885250,19.241000,59.388750,14.068750,0.882500,111.977000,-2.629000,1.810000,2.841000,77.550000,...,-1.575000,123.017500,0.000000,44.422500,-1.342000,-1.584000,37.312000,-1.612000,-1.576500,0.000000
50%,112.406000,46.524000,73.752000,38.467500,2.107000,225.885000,-0.244000,2.491500,12.315500,120.664500,...,-0.024000,179.739000,1.000000,65.561000,0.000000,-0.033000,47.902000,-0.010000,-0.002000,40.512500
75%,135.482000,73.598000,92.259000,79.169000,3.690000,478.226000,0.958000,2.961000,27.591000,200.478250,...,1.561000,263.379250,2.000000,103.342000,1.336000,1.562000,66.637000,1.589500,1.576000,109.933750
max,1192.026000,690.075000,1349.351000,2834.999000,8.503000,4974.979000,16.690000,5.684000,2834.999000,1852.462000,...,3.142000,2003.976000,3.000000,1120.573000,4.499000,3.141000,721.456000,4.500000,3.142000,1633.433000


# Train model with given hyperparameters

In [8]:
from proj1_helpers import *
DATA_TRAIN_PATH = os.path.join(os.getcwd(), '../data/train.csv')
y, tX, ids = load_csv_data(DATA_TRAIN_PATH)

preprocessing = Preprocessing()
y[y==-1] = 0
y = y.astype(np.int)
preprocessing.replace_outliers_by_nan(tX)

In [9]:
#Delete rows from tX and y, which has more than 8 nan values in tX and 0 label in y
num_nan_in_row = np.isnan(tX).sum(axis=1)
mask = (y==0) & (num_nan_in_row>8)
y = y[~mask]
tX = tX[~mask]

#Create train and validate sets
num_examples = tX.shape[0]
val_size = 10000
train_samples = np.random.choice(num_examples, replace=False, size=val_size)
mask_val = np.isin(np.arange(tX.shape[0]), train_samples)
x_val, y_val = tX[mask_val], y[mask_val]
x_train, y_train = tX[~mask_val], y[~mask_val]

In [10]:
use_transformation = False
handling_outlier = 'predict'
lr = 0.1
lambda_ = 100
degree = np.arange(2, 5)
epochs_preprocess = 10
epochs = 50
batch_size = 32
units = 1
activation = 'sigmoid'

In [12]:
preprocessing = Preprocessing(
    use_transformations=use_transformation,
    handling_outliers=handling_outlier,
)
train_data, models = preprocessing.preprocess(
    x_train,
    lr=lr,
    lambda_=lambda_,
    batch_size=batch_size,
    epochs=epochs,
    degrees=degree
)

model = NNModel(train_data.shape[1])
model.add_layer(units=units, activation=activation)
model.train(train_data, y_train, lr=lr,
            lambda_=lambda_,
            batch_size=batch_size, epochs=epochs,
            verbose=1)

preprocessing = Preprocessing(
    use_transformations=use_transformation,
    handling_outliers = handling_outlier,
)
val_data, _ = preprocessing.preprocess(
    x_val,
    train=False,
    models=models,
    degrees=degree
)

y_pred = model.predict(val_data)
y_pred = y_pred > 0.5
y_pred = y_pred.squeeze()
accuracy = (y_pred==y_val).mean()
print("Validation arruracy is", accuracy)

Preprocesing started!

[[4.35157936e-04 8.59632332e-01 1.76750908e+00 ... 0.00000000e+00
  1.00000000e+00 0.00000000e+00]
 [1.16103275e-09 2.20675604e+04 2.55959438e+00 ... 1.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [7.17331933e-06 1.01090913e+01 2.55959438e+00 ... 1.00000000e+00
  0.00000000e+00 0.00000000e+00]
 ...
 [2.73713058e-48 2.70517611e+00 1.98021339e-03 ... 0.00000000e+00
  0.00000000e+00 1.00000000e+00]
 [1.51452501e-03 1.78126601e-34 1.66079947e+00 ... 0.00000000e+00
  1.00000000e+00 0.00000000e+00]
 [5.65277906e-07 6.97495144e-22 6.98481874e-15 ... 1.00000000e+00
  0.00000000e+00 0.00000000e+00]]
Training started
>Epoch #50:	[####################]; Loss: nan
Training ended

Training started
>Epoch #50:	[####################]; Loss: nan
Training ended

Training started
>Epoch #33:	[#############       ]; Loss: nan

KeyboardInterrupt: 

In [ ]:
pred = model.predict(train_data)

In [ ]:
res = pred>0.5

In [ ]:
res = res.squeeze()

In [ ]:
pred = np.zeros(y_train.shape)

In [ ]:
pred[res] = 1

In [ ]:
np.mean(pred==y_train)

## Generate predictions and save ouput in csv format for submission:

In [14]:
DATA_TEST_PATH = DATA_TRAIN_PATH = os.path.join(os.getcwd(), '../data/test.csv')
_, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)

In [15]:
preprocessing = Preprocessing(
    use_transformations=use_transformation,
    handling_outliers = handling_outlier,
)
test_data, _ = preprocessing.preprocess(
    tX_test,
    train=False,
    models=models,
    degrees=degree
)

Preprocesing started!

Preprocessing ended



In [16]:
OUTPUT_PATH = 'prediction.csv' # TODO: fill in desired name of output file for submission
y_pred = model.predict(test_data)
res = y_pred>0.5
res = res.squeeze()
pred = -np.ones(res.shape)
pred[res] = 1
create_csv_submission(ids_test, pred, OUTPUT_PATH)